In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/envs/competition_effdet/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [9]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_x101_64x4d_fpn_1x_coco_custom.py')

root='../../dataset/'

# cfg.resume_from = './work_dirs/AlbuAll-RBCMixupGaussian_faster_rcnn_r50_fpn_1x_trash/epoch_6.pth'


# dataset config 수정
cfg.data.train.dataset.classes = classes
cfg.data.train.dataset.img_prefix = root
cfg.data.train.dataset.ann_file = root + '5___train_MultiStfKFold.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize
# cfg.data.train.pipeline[0]['img_scale'] = (1024,1024) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + '5___val_MultiStfKFold.json' # test json 정보
# cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize


cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/resNext_fold5_faster_rcnn_x101_64x4d_fpn_1x_coco_custom.py'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.evaluation = dict(save_best='bbox_mAP_50', metric='bbox')
cfg.checkpoint_config = dict(max_keep_ckpts=1, interval=1)
cfg.device = get_device()

In [10]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]
# datasets_val = [build_dataset(cfg.data.val)]

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


/opt/conda/envs/competition_effdet/lib/python3.7/site-packages/albumentations/augmentations/dropout/cutout.py:51: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  FutureWarning,


In [11]:
# dataset 확인
datasets[0]

In [12]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-11-30 03:13:26,262 - mmcv - INFO - initialize ResNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnext101_64x4d'}
2022-11-30 03:13:26,264 - mmcv - INFO - load model from: open-mmlab://resnext101_64x4d
2022-11-30 03:13:26,265 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://resnext101_64x4d
2022-11-30 03:13:26,607 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-11-30 03:13:26,634 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-11-30 03:13:26,643 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-11-30 03:13:26,748 - mmcv - INFO - 
backbone.conv1.weight 

In [13]:
# cfg.log_config.hooks = [dict(type='TextLoggerHook'),
#                         dict(type='WandbLoggerHook',
#                              init_kwargs=dict(project='trash_detection',
#                                               name=f'fasterrcnn_Ver.0', #저장할 이름
#                                               entity='aidatapkt'))
#                        ]

In [14]:
import datetime
now = (datetime.datetime.now().replace(microsecond=0) + datetime.timedelta(hours=9)).strftime("%m-%d %H:%M")
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={
            'project': 'trash_detection',
            'entity': 'aidatapkt',
            'name' : f'resnext_fold5_fasterRcnn_{cfg.model.type}_{now}_pkt',
            'tags': [cfg.model.backbone.type, cfg.model.neck.type, 'mutlscale 512~102, 64'
                     # str(cfg.data.train.pipeline[2]['img_scale'][0])
                    ] 
            },
         interval=10,
         # log_checkpoint=False,
         # log_checkpoint_metadata=False,
         # num_eval_images=10,
         # bbox_score_thr=0.3
        )]

In [15]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2022-11-30 03:13:27,172 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-11-30 03:13:27,198 - mmdet - INFO - Start running, host: root@7ec8fd27c771, work_dir: /opt/ml/baseline/mmdetection/work_dirs/resNext_fold5_faster_rcnn_x101_64x4d_fpn_1x_coco_custom.py
2022-11-30 03:13:27,199 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 ------

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2022-11-30 03:14:50,147 - mmdet - INFO - Epoch [1][50/977]	lr: 9.890e-06, eta: 8:24:59, time: 1.555, data_time: 0.068, memory: 18203, loss_rpn_cls: 0.5797, loss_rpn_bbox: 0.1076, loss_cls: 1.0846, acc: 78.6914, loss_bbox: 0.0874, loss: 1.8593, grad_norm: 16.6946
2022-11-30 03:16:02,806 - mmdet - INFO - Epoch [1][100/977]	lr: 1.988e-05, eta: 8:07:15, time: 1.453, data_time: 0.011, memory: 18203, loss_rpn_cls: 0.2417, loss_rpn_bbox: 0.0910, loss_cls: 0.3997, acc: 92.2393, loss_bbox: 0.2677, loss: 1.0000, grad_norm: 6.2270
2022-11-30 03:17:15,851 - mmdet - INFO - Epoch [1][150/977]	lr: 2.987e-05, eta: 8:01:22, time: 1.461, data_time: 0.011, memory: 18203, loss_rpn_cls: 0.1959, loss_rpn_bbox: 0.0915, loss_cls: 0.4482, acc: 90.7676, loss_bbox: 0.3316, loss: 1.0672, grad_norm: 6.0693
2022-11-30 03:18:27,918 - mmdet - INFO - Epoch [1][200/977]	lr: 3.986e-05, eta: 7:56:14, time: 1.441, data_time: 0.011, memory: 18203, loss_rpn_cls: 0.1695, loss_rpn_bbox: 0.0737, loss_cls: 0.4369, acc: 91.0312,

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 977/977, 6.7 task/s, elapsed: 146s, ETA:     0s

2022-11-30 03:40:11,422 - mmdet - INFO - Evaluating bbox...


...

2022-11-30 12:09:06,112 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.23s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.24s).
Accumulating evaluation results...


2022-11-30 12:09:10,732 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.372
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.537
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.448
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.528
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.528
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.528
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.067
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.217
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.06s).


learning_rate,▃███████████████▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▄▁▃▃▃▃▄▄▃▅▄▄▅▄▄▅▇▇▆▄▇▆▆▇▆▆▇▆▇▆█▆▇▇▆▇▆▇▇█
train/grad_norm,█▅▃▂▂▂▂▃▂▂▂▂▂▂▃▃▁▁▂▃▂▂▃▂▂▃▂▂▂▂▂▃▃▂▃▂▃▃▂▂
train/loss,██▆▅▅▅▅▄▅▃▄▄▄▄▄▄▂▁▃▄▂▂▃▂▃▂▂▃▂▂▁▃▂▂▂▁▂▂▂▁
train/loss_bbox,▆█▆▆▆▆▅▄▅▃▄▄▄▅▅▄▃▁▃▅▃▃▃▂▄▃▃▃▂▃▂▄▂▂▃▂▃▂▃▁
train/loss_cls,▇█▆▆▆▆▅▅▅▄▄▄▄▅▄▄▂▂▃▄▃▂▃▂▃▃▂▂▂▂▁▂▂▂▂▁▂▂▂▁
train/loss_rpn_bbox,██▆▅▇▅▄▃▅▄▆▃▄▅▃▃▂▂▄▅▃▃▂▃▄▂▂▄▂▃▂▄▄▂▃▁▄▂▃▂
train/loss_rpn_cls,█▅▃▃▃▃▂▂▂▂▃▂▂▂▂▂▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▂▁▁
val/bbox_mAP,▁▂▄▄▅▅▅▅▇▇▇█████████
val/bbox_mAP_50,▁▂▄▅▅▆▆▆▇▇██████████
